In [10]:
import pandas as pd
from assign_unique_ids_functions import *
import inspect

In [11]:
uhd = pd.read_csv("input/unit-history_demographics.csv.gz")
amd = pd.read_csv("input/all-members_demographics.csv.gz")
asd = pd.read_csv("input/all-sworn_demographics.csv.gz")
asud = pd.read_csv("input/all-sworn-units_demographics.csv.gz")

In [ ]:
col_vals = {
    'First.Name': 10,
    'Last.Name': 9,
    'Middle.Initial': 7,
    'Suffix.Name': 5,
    'Appointed.Date': 9,
    'Gender': 7,
    'Race': 6,
    'Birth.Year': 8,
    'Current.Age': 7,
    'Current.Age.m1': 7,
    'Current.Age.p1': 7,
    'F4FN': 9,
    'F4LN': 9
}

In [12]:
uhd.head()

,unit_history_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Current.Age,Gender,Race,Star1,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Star10,Current.Unit
0,1,JEFFERY,AARON,NaN,NaN,2005-09-26,44,MALE,WHITE,13001,-999,-999,-999,-999,-999,-999,-999,-999,-999,14.0
1,2,KARINA,AARON,NaN,NaN,2005-09-26,35,FEMALE,HISPANIC,17545,13705,-999,-999,-999,-999,-999,-999,-999,-999,15.0
2,3,DANIEL,ABATE,NaN,NaN,1970-06-15,74,MALE,WHITE,14535,3023,2628,-999,-999,-999,-999,-999,-999,-999,543.0
3,4,ANTHONY,ABBATE,NaN,NaN,1994-12-05,47,MALE,WHITE,18601,-999,-999,-999,-999,-999,-999,-999,-999,-999,20.0
4,5,CARMEL,ABBATE,NaN,NaN,1969-01-06,73,MALE,WHITE,20484,-999,-999,-999,-999,-999,-999,-999,-999,-999,640.0


In [7]:
amd.head()

,all_members_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race
0,1,JEFFERY,AARON,M,NaN,2005-09-26,1971,MALE,WHITE
1,2,KARINA,AARON,NaN,NaN,2005-09-26,1980,FEMALE,HISPANIC
2,3,DANIEL,ABATE,P,NaN,1970-06-15,1942,MALE,WHITE
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,1968,MALE,WHITE
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,1942,MALE,WHITE


In [8]:
asd.head()

,all_sworn_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race
0,1,JEFFERY,AARON,M,NaN,2005-09-26,1971,MALE,WHITE
1,2,KARINA,AARON,NaN,NaN,2005-09-26,1980,FEMALE,HISPANIC
2,3,DANIEL,ABATE,P,NaN,1970-06-15,1942,MALE,WHITE
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,1968,MALE,WHITE
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,1942,MALE,WHITE


In [9]:
asud.head()

,all_sworn_units_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race,Current.Unit
0,1,JEFFERY,AARON,M,NaN,2005-09-26,1971,MALE,WHITE,3.0
1,2,KARINA,AARON,NaN,NaN,2005-09-26,1980,FEMALE,HISPANIC,15.0
2,3,DANIEL,ABATE,P,NaN,1970-06-15,1942,MALE,WHITE,543.0
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,1968,MALE,WHITE,20.0
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,1942,MALE,WHITE,640.0


In [ ]:
[
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age"],
]

In [5]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2'], return_unmatched = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmatched:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [ ]:
def StarMerge(df1, df2, on_list, keep_columns, return_unmerged = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    df2 = df2.loc[(df2["Star"].notnull()) & (df2["Star"] > 0)]
    stars = ["Star" + str(i) for i in range(1,11)]
    for mc_cols in on_list:
        for star in stars:
            mc1 = mc_cols + [star]
            mc2 = mc_cols + ["Star"]
            df1t = RemoveDuplicates(df1.loc[(df1[star].notnull()) & (df1[star] > 0), keep_columns[:1] + mc1], mc1)
            df2t = RemoveDuplicates(df2[keep_columns[1:] + mc2], mc2)
            dfmt = df1t.merge(df2t, left_on = mc1, right_on = mc2, how='inner')
            if dfmt.shape[0] > 0:
                print('******')
                print(mc1)
                print(dfmt.shape[0])
                print('******')
                dfmt['Match'] = '-'.join(mc + [star + "/" + "Star"])
                dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
                df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
                df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmerged:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)